# Итак, что же такое «декоратор»?
вот пример того, как работают декораторы:

In [3]:
def makebold(fn):
    def wrapped():
        return "<b>" + fn() + "</b>"
    return wrapped
 
def makeitalic(fn):
    def wrapped():
        return "<i>" + fn() + "</i>"
    return wrapped
 
@makebold
@makeitalic
def hello():
    return "hello -_-"

# foo2 = makeitalic(hello)
 
print (hello())
# print(foo2())

<b><i>hello -_-</i></b>


# Функции в Python'e являются объектами

Для того, чтобы понять, как работают декораторы, в первую очередь следует осознать, что в Python'е функции — это тоже объекты.
Давайте посмотрим, что из этого следует:

In [4]:
def shout(word="да"):
    return word.capitalize()+"!"

In [5]:
help(str.capitalize)

Help on method_descriptor:

capitalize(self, /)
    Return a capitalized version of the string.
    
    More specifically, make the first character have upper case and the rest lower
    case.



In [6]:
print (shout())

Да!


In [7]:
# Так как функция - это объект, вы связать её с переменнной,
# как и любой другой объект
scream = shout

In [8]:
print (scream())

Да!


In [ ]:
# Более того, это значит, что мы можем удалить "shout", и функция всё ещё
# будет доступна через переменную "scream"
 
del shout
try:
    print (shout())
except NameError as e:
    print (e)
    #выведет: "name 'shout' is not defined"

In [ ]:
print (scream())

In [ ]:
def talk():
    # Внутри определения функции "talk" мы можем определить другую...
    def whisper(word="да"):
        return word.lower()+"...";
 
    # ... и сразу же её использовать!
    print (whisper())
    # Теперь, КАЖДЫЙ РАЗ при вызове "talk", внутри неё определяется а затем

# и вызывается функция "whisper".
talk()
# выведет: "да..."
 
# Но вне функции "talk" НЕ существует никакой функции "whisper":
try:
    print (whisper())
except NameError as  e:
    print (e)
    #выведет : "name 'whisper' is not defined"

# Ссылки на функции

Ну что, вы всё ещё здесь?:)

Теперь мы знаем, что функции являются полноправными объектами, а значит:
- могут быть связаны с переменной;
- могут быть определены одна внутри другой.

Что ж, а это значит, что одна функция может вернуть другую функцию!
Давайте посмотрим:

In [ ]:
def getTalk(t="shout"):
 
    # Мы определяем функции прямо здесь
    def shout(word="да"):
        return word.capitalize()+t+"!"
 

    def whisper(word="да") :
        return word.lower()+" "+t+"...";
 
    # Затем возвращаем необходимую
    if t == "shout":
        # Заметьте, что мы НЕ используем "()", нам нужно не вызвать функцию,
        # а вернуть объект функции
        return shout
    else:
        return whisper

In [ ]:
# Как использовать это непонятное нечто?
# Возьмём функцию и свяжем её с переменной
talk = getTalk()

In [ ]:
# Как мы можем видеть, "talk" теперь - объект "function":
print (talk)
# выведет: <function shout at 0x0000026F01ED18C8>

In [ ]:
# Который можно вызывать, как и функцию, определённую "обычным образом":
print (talk())

In [ ]:
# Если нам захочется - можно вызвать её напрямую из возвращаемого значения:
print (getTalk("dfgsfdgsdfg")())
# выведет: да...

Подождите, раз мы можем возвращать функцию, значит, мы можем и передавать её другой функции, как параметр:

In [ ]:
def doSomethingBefore(func):
    print ("Я делаю что-то ещё, перед тем как вызвать функцию, которую ты мне передал")
    print (func())
    
doSomethingBefore(scream)
#выведет:
# Я делаю что-то ещё, перед тем как вызвать функцию, которую ты мне передал
# Да!

Ну что, теперь у нас есть все необходимые знания для того, чтобы понять, как работают декораторы.
Как вы могли догадаться, декораторы — это, по сути, просто своеобразные «обёртки», которые **дают нам возможность делать что-либо до и после того, что сделает декорируемая функция, не изменяя её.**
# Создадим свой декоратор «вручную»

In [ ]:
# Декоратор - это функция, ожидающая ДРУГУЮ функцию в качестве параметра
def my_shiny_new_decorator(a_function_to_decorate):
    # Внутри себя декоратор определяет функцию-"обёртку".
    # Она будет (что бы вы думали?..) обёрнута вокруг декорируемой,
    # получая возможность исполнять произвольный код до и после неё.

    def the_wrapper_around_the_original_function():
        # Поместим здесь код, который мы хотим запускать ДО вызова
        # оригинальной функции
        print ("Я - код, который отработает до вызова функции")
 
        # ВЫЗОВЕМ саму декорируемую функцию
        a_function_to_decorate()

        # А здесь поместим код, который мы хотим запускать ПОСЛЕ вызова
        # оригинальной функции
        print ("А я - код, срабатывающий после")

    # На данный момент функция "a_function_to_decorate" НЕ ВЫЗЫВАЛАСЬ НИ РАЗУ

    # Теперь, вернём функцию-обёртку, которая содержит в себе
    # декорируемую функцию, и код, который необходимо выполнить до и после.
    # Всё просто!
    return the_wrapper_around_the_original_function

def a_stand_alone_function():
    print ("Я простая одинокая функция, ты ведь не посмеешь меня изменять?..")
    
a_stand_alone_function()

In [ ]:
a_stand_alone_function_decorated = my_shiny_new_decorator(a_stand_alone_function)
a_stand_alone_function_decorated()

Наверное, теперь мы бы хотели, чтобы каждый раз, во время вызова a_stand_alone_function, вместо неё вызывалась a_stand_alone_function_decorated. Нет ничего проще, просто перезапишем a_stand_alone_function функцией, которую нам вернул my_shiny_new_decorator:

In [ ]:
a_stand_alone_function = my_shiny_new_decorator(a_stand_alone_function)
a_stand_alone_function()

Вы ведь уже догадались, что это ровно тоже самое, что делают @декораторы.:)

In [ ]:
@my_shiny_new_decorator
def another_stand_alone_function():
    print ("Оставь меня в покое")

another_stand_alone_function()

Да, всё действительно так просто! @decorator — просто синтаксический сахар для конструкций вида:

Конечно, можно вкладывать декораторы друг в друга, например так:

In [ ]:
def bread(func):
    def wrapper():
        print ("</------\>")
        func()
        print ("<\______/>")
    return wrapper
 
def ingredients(func):
    def wrapper():
        print ("#помидоры#")
        func()
        print("~салат~")
    return wrapper
 
def sandwich(food="--ветчина--"):
    print (food)

sandwich()
#выведет: --ветчина--
sandwich = bread(ingredients(sandwich))
sandwich()
#выведет:
# </------\>
# #помидоры#
# --ветчина--
# ~салат~
# <\______/>

И используя синтаксис декораторов:

In [ ]:
@bread
@ingredients
def sandwich(food="--ветчина--"):
    print (food)

sandwich()
#выведет:
# </------\>
# #помидоры#
# --ветчина--
# ~салат~
# <\______/>

# Как можно использовать декораторы?

И в заключение, я бы хотел ответить на вопрос, который я часто слышу: зачем же нужны декораторы? Как их можно использовать?
Декораторы могут быть использованы для расширения возможностей функций из сторонних библиотек (код которых мы не можем изменять), или для упрощения отладки (мы не хотим изменять код, который ещё не устоялся).
Так же полезно использовать декораторы для расширения различных функций одним и тем же кодом, без повторного его переписывания каждый раз, например:

In [ ]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло
    выполнение декорируемой функции.
    """
    import time
    def wrapper(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        print (func.__name__, time.clock() - t)
        return res
    return wrapper

def logging(func):
    """
    Декоратор, логирующий работу кода.
    (хорошо, он просто выводит вызовы, но тут могло быть и логирование!)
    """
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print (func.__name__, args, kwargs)
        return res
    return wrapper


def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов
    декорируемой функции.
    """
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        res = func(*args, **kwargs)
        print ("{0} была вызвана: {1}x".format(func.__name__, wrapper.count))
        return res
    wrapper.count = 0
    return wrapper


@benchmark
@logging
@counter
def reverse_string(string):
    return string[::-1]
 
print (reverse_string("А роза упала на лапу Азора"))
# print (reverse_string("A man, a plan, a canoe, pasta, heros, rajahs, a coloratura, maps, snipe, percale, macaroni, a gag, a banana bag, a tan, a tag, a banana bag again (or a camel), a crepe, pins, Spam, a rut, a Rolo, cash, a jar, sore hats, a peon, a canal: Panama!"))


In [ ]:
print (reverse_string("А роза упала на лапу Азора"))

In [ ]:
def fun(a1,a2,a3,a4,a5, **kwargs):
    print(kwargs)
#     print(args)
    return sum([a1,a2,a3,a4,a5])

In [ ]:
a = [1,2,3,4,5]
b = {
    'ASSSd':123,
    '44':3
}
fun(*a, **b)

In [ ]:
a1,a2,a3,a4 = a
print(a)
pr